# Paper Roll Cut

Cristina Hernández, Beatriz Jimenez, Macarena Vargas, Guillermo Ruiz

In [34]:
import pyomo.environ as pe
import pyomo.opt as po
from pyomo.environ import NonNegativeReals, Binary

#### Create the model 

In [35]:
model = pe.ConcreteModel()

#### Sets


${c}$ : Orders = {1, 2, ..., 10}

In [36]:
model.c = pe.Set(initialize = [1,2,3,4,5,6,7,8,9,10])

${n}$ : Master paper rollsSubset of Refineries  = {1,2,3,...,N}

Tenemoss 20 rollos de tipo 1, 30 de tipo 2, 15 de tipo 3, etc., sumamos todos los pedidos: N = 20 + 30 + 15 + ... + 5 = 240. Selecionamos 240 como el numero de master paper rolls disponibles.

In [37]:
model.n = pe.Set(initialize = range(1, 240 + 1))

#### Parameters
$R_{c}$: Number of rolls in order $c$ [units] 


In [38]:
pedido_unidades = {1: 20, 2: 30, 3: 15, 4: 25, 5: 40, 6: 25, 7: 50, 8: 15, 9: 10, 10: 5}
model.R = pe.Param(model.c, initialize=pedido_unidades)

$L_{c}$: Length of order $c$ [m] 

In [39]:
pedido_longitudes = {1: 0.50, 2: 0.75, 3: 1.00, 4: 1.25, 5: 1.50, 6: 1.75, 7: 2.00, 8: 2.25, 9: 2.50, 10: 2.75}
model.L = pe.Param(model.c, initialize=pedido_longitudes)

$L_{n}$: Maximum length of waste in roll $n$ (2.5m)

In [40]:
max_waste = 2.5
model.Ln = pe.Param(model.n, initialize={n: max_waste for n in model.n})

$QL$: Length of master roll [m]

In [41]:
model.QL = pe.Param(initialize=3.0)

$QP$: Maximum number of pieces pe rmaster rolll [units]

In [42]:
model.QP = pe.Param(initialize=5)

#### Variables
$y_{n}$: Master papear roll $n$ is in use {0,1} 


In [43]:
model.y = pe.Var(model.n, within=pe.Binary)

$z_{c,n}$: Number of orders $c$ in paper roll $n$ {0, 1, 2, 3, 4, 5}

In [44]:
model.z = pe.Var(model.c, model.n, within=pe.NonNegativeIntegers)

$l_{n}$: Length of waste in paper roll $n$ [m]

In [45]:
model.l = pe.Var(model.n, within=pe.NonNegativeReals)

$\delta_{n}$: Waste in paper roll $n$ {0,1} 

In [46]:
model.d = pe.Var(model.n, within=pe.Binary)

#### Objective Function

min $\sum_{n}y_n$ or min $\sum_{n}l_n$

In [47]:
def objetivo_min_rollos(model):
    return sum(model.y[n] for n in model.n)

model.obj = pe.Objective(rule=objetivo_min_rollos, sense=pe.minimize)

#### Constraints

Constraint 1: Maximum length of master paper roll n [m]

$\sum_c L_c ·z_{c,n} + l_n = QL ·y_n \quad \forall n$

In [48]:
def capacidad_rule(model, n):
    return sum(model.L[c] * model.z[c, n] for c in model.c) + model.l[n] == model.QL * model.y[n]
model.capacidad = pe.Constraint(model.n, rule=capacidad_rule)



Constraint 2: Maximum number of cuts in master paper roll n [units]

$\sum_c z_{c,n} + \delta_n \leq QP·y_n\quad \forall n$

In [49]:
def max_cortes_rule(model, n):
    return sum(model.z[c, n] for c in model.c) <= model.QP * model.y[n]
model.max_cortes = pe.Constraint(model.n, rule=max_cortes_rule)

Constraint 3: Meet clients’ order c [units]

$\sum _n z_{c,n}=R_c\quad \forall c$

In [50]:
def demanda_rule(model, c):
    return sum(model.z[c, n] for n in model.n) == model.R[c]
model.demanda = pe.Constraint(model.c, rule=demanda_rule)

Constraint 4: Waste piece in master paper roll n [m]

$l_n \leq \delta _n·2.5 \quad \forall n$

In [51]:
def max_residuo_rule(model, n):
    return model.l[n] <= model.d[n]*model.Ln[n]
model.max_residuo = pe.Constraint(model.n, rule=max_residuo_rule)

Constraint 5: Positive waste in paper roll n [m]

$l_n \geq 0 \quad \forall n $

In [52]:
def residuo_positivo_rule(model,n):
    return model.l[n]>= 0
model.residuo_positivo=pe.Constraint(model.n, rule =residuo_positivo_rule)

Constraint 6: for all the master paper rolls in which an order type 1 is obtained, the same roll must provide one order type 5 or type 9.

$z_{5,n}+z_{9,n} \geq \frac {z_{1,n}} {QP}$



In [53]:
def type1_rule(model, n):
    return model.z[5, n] + model.z[9, n] >= model.z[1, n] / model.QP
model.type1 = pe.Constraint(model.n, rule=type1_rule)

#### Solve model

In [55]:
solver = pe.SolverFactory('gurobi')
results = solver.solve(model, tee=True)

# Mostrar estado de la optimización
print("Solver Status:", results.solver.status)
print("Termination Condition:", results.solver.termination_condition)

Set parameter Username
Set parameter LicenseID to value 2703329
Academic license - for non-commercial use only - expires 2026-09-04
Read LP format model from file C:\Users\macar\AppData\Local\Temp\tmpfzykwgp7.pyomo.lp
Reading time = 0.06 seconds
x1: 1210 rows, 3120 columns, 9360 nonzeros
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1210 rows, 3120 columns and 9360 nonzeros
Model fingerprint: 0xe313e6ca
Variable types: 240 continuous, 2880 integer (480 binary)
Coefficient statistics:
  Matrix range     [2e-01, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+00, 5e+01]
Presolve removed 480 rows and 240 columns
Presolve time: 0.05s
Presolved: 730 rows, 2880 columns, 8640 nonzeros
Variable types: 0 continuous, 2880 intege

In [56]:
optimal_rolls = sum(pe.value(model.y[n]) for n in model.n)
print("Optimal number of paper rolls:", int(optimal_rolls))

Optimal number of paper rolls: 127


In [57]:
print("SCHEDULE DE CORTES:")
for n in model.n:
    if pe.value(model.y[n]) > 0.5:  # Si el rollo grande n se usa
        print(f"\nMaster roll n={n} usado:")
        for c in model.c:
            cortes = pe.value(model.z[c, n])
            if cortes > 0.5:
                print(f"  - Se cortan {int(round(cortes))} rollos de tipo {c} (longitud {pe.value(model.L[c])} m)")
        print(f"  Residuo final: {pe.value(model.l[n]):.2f} m")

SCHEDULE DE CORTES:

Master roll n=1 usado:
  - Se cortan 1 rollos de tipo 4 (longitud 1.25 m)
  - Se cortan 1 rollos de tipo 6 (longitud 1.75 m)
  Residuo final: 0.00 m

Master roll n=2 usado:
  - Se cortan 1 rollos de tipo 4 (longitud 1.25 m)
  - Se cortan 1 rollos de tipo 6 (longitud 1.75 m)
  Residuo final: 0.00 m

Master roll n=3 usado:
  - Se cortan 1 rollos de tipo 4 (longitud 1.25 m)
  - Se cortan 1 rollos de tipo 6 (longitud 1.75 m)
  Residuo final: 0.00 m

Master roll n=4 usado:
  - Se cortan 1 rollos de tipo 4 (longitud 1.25 m)
  - Se cortan 1 rollos de tipo 6 (longitud 1.75 m)
  Residuo final: 0.00 m

Master roll n=5 usado:
  - Se cortan 1 rollos de tipo 4 (longitud 1.25 m)
  - Se cortan 1 rollos de tipo 6 (longitud 1.75 m)
  Residuo final: 0.00 m

Master roll n=6 usado:
  - Se cortan 1 rollos de tipo 4 (longitud 1.25 m)
  - Se cortan 1 rollos de tipo 6 (longitud 1.75 m)
  Residuo final: 0.00 m

Master roll n=7 usado:
  - Se cortan 1 rollos de tipo 4 (longitud 1.25 m)
  - S

In [ ]:
from collections import Counter
combinaciones = []
for n in model.n:
    if pe.value(model.y[n]) > 0.5:
        cortes = tuple(int(round(pe.value(model.z[c, n]))) for c in model.c)
        combinaciones.append(cortes)
conteo = Counter(combinaciones)
print("Resumen de combinaciones de cortes usadas:")
for combo, cantidad in conteo.items():
    # Mostrar solo los tipos de corte que realmente se usaron en la combinación
    cortes_str = " y ".join(
        f"{cant} de tipo {c}" for c, cant in zip(model.c, combo) if cant > 0
    )
    print(f"{cantidad} rollo(s) con: {cortes_str}")

Resumen de combinaciones de cortes usadas:
25 rollo(s) con: 1 de tipo 4 y 1 de tipo 6
15 rollo(s) con: 1 de tipo 3 y 1 de tipo 7
20 rollo(s) con: 1 de tipo 7
15 rollo(s) con: 1 de tipo 2 y 1 de tipo 7
15 rollo(s) con: 1 de tipo 2 y 1 de tipo 8
3 rollo(s) con: 3 de tipo 1 y 1 de tipo 5
18 rollo(s) con: 2 de tipo 5
10 rollo(s) con: 1 de tipo 1 y 1 de tipo 9
5 rollo(s) con: 1 de tipo 10
1 rollo(s) con: 1 de tipo 1 y 1 de tipo 5


In [63]:
model.pprint()

2 Set Declarations
    c : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    n : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :  240 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 15

In [64]:
model.display() 

Model unknown

  Variables:
    y : Size=240, Index=n
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   1.0 :     1 : False : False : Binary
          2 :     0 :   1.0 :     1 : False : False : Binary
          3 :     0 :   1.0 :     1 : False : False : Binary
          4 :     0 :   1.0 :     1 : False : False : Binary
          5 :     0 :   1.0 :     1 : False : False : Binary
          6 :     0 :   1.0 :     1 : False : False : Binary
          7 :     0 :   1.0 :     1 : False : False : Binary
          8 :     0 :   1.0 :     1 : False : False : Binary
          9 :     0 :   1.0 :     1 : False : False : Binary
         10 :     0 :   1.0 :     1 : False : False : Binary
         11 :     0 :   1.0 :     1 : False : False : Binary
         12 :     0 :   1.0 :     1 : False : False : Binary
         13 :     0 :   1.0 :     1 : False : False : Binary
         14 :     0 :   1.0 :     1 : False : False : Binary
         15 :     0 :   1.0 :  